# <center>Attrition Analysis Sample</center>

We will use data/analysis from https://www.ibm.com/communities/analytics/watson-analytics-blog/hr-employee-attrition/ to demonstrate attrition models. Please remember this is a statistical model and does not consider the events or the temporal sequence that led to attrition. Most analyes use both quantitative methods as well as survival models.

## 1. Download data

In [1]:
# Download the data
uri = 'https://community.watsonanalytics.com/wp-content/uploads/2015/03/WA_Fn-UseC_-HR-Employee-Attrition.xlsx'
attrition_pd = pd.read_excel(uri)
attrition_pd

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,3,Male,41,4,2,Laboratory Technician,4,Married,2571,12290,4,Y,No,17,3,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,4,Male,42,2,3,Healthcare Representative,1,Married,9991,21457,4,Y,No,15,3,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,2,Male,87,4,2,Manufacturing Director,2,Married,6142,5174,1,Y,Yes,20,4,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,4,Male,63,2,2,Sales Executive,2,Married,5390,13243,2,Y,No,14,3,4,80,0,17,3,2,9,6,0,8


## 2. Explore data

In [2]:
#Print attributes of the data
pd.DataFrame(
    zip(attrition_pd.columns, attrition_pd.dtypes),
    columns=['Attribute', 'Data Type']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
Attribute,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Data Type,int64,object,object,int64,object,int64,int64,object,int64,int64,int64,object,int64,int64,int64,object,int64,object,int64,int64,int64,object,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64


The data contains 
* Attrition -- Label attribute (predicted variable)
* Age -- Continuous variable  (are older employees more likely to attrite than younger employees?)
* Business Travel -- Frequency of travel; are road-warriors more likely to attrite than others?
* Daily Rate -- Are compensated employees more satisfied?
* Department -- does the field of employment make a difference?
* Distance from Home -- is long haul commute a likely indicator for employee frustration?
* Education level -- An ordinal rating for level of education. Is education offering better opportunities to lure employees away?
* Education Field -- Is academic field an indicator for market appetite of skills?
* Employee Count -- duh -- a count of 1 employee is 1. Please ignore this attribute.
* Employee Number -- Employee ID; ignore since this is a apriori ID and has no bearing on the unless an employee attrites because they do not like their number. More likely this is an ID attribute and deserves to be thrown out.
* Environment Satisfaction -- A score for employee satisfaction from previous survey -- the company's environment in general. Of course, higher the satisfaction, the lesser the attrition.
* Gender -- is gender likely a factor? Does gender contribute to churn/volatile qualities?
* Hourly Rate -- Are compensated employees more satisfied? This is of course redundant in view of the daily rate and monthly rate. How do we consolidate all these measures of the compensation into a singular attribute?
* Job Involvement -- How hands on are the employees? Are involved employees more satisfied?
* Job Level -- do executives face different pressures that forces involuntary attrition?
* Job Role -- Does title make a difference? This is probably redundant considering department and the job title; but albeit let us keep it in analysis
* Job Satisfaction -- past satisfaction score from survey about their specific job. I guess this factor can be confounded with environment satisfaction because it is hard to isolate one's job from company overall.
* MaritalStatus -- Do single individuals more likely to attritte because of the socio-econo-emotional independence?
* Monthly Income -- How is this different than monthly rate or daily rate or hourly rate? Not sure. Let us leave it in.
* Monthly Rate --  -- Are compensated employees more satisfied? This is of course redundant in view of the daily rate and hourly rate. How do we consolidate all these measures of the compensation into a singular attribute?
* NumCompaniesWorked -- how many companies in the past have employees rotated? This is meaningless unless we also take into account the total experience duration into account.
* Over18 -- is the employee an adult? All employees are adults. So it is best to ignore because this attribute is poorly ranked.
* OverTime -- is the employee exempt or eligible for overtime. I suppose exempt employees are fulltime and therefore less likely to attrite than those that rely on variable/non-exempt compensation.
* PercentSalaryHike -- hike in percentage during last performance check-in.
* PerformanceRating -- was employee rated failing, mediocre, or stellar?
* Relationship Satisfaction -- score from last satisfaction survey wrt to relationship with the team.
* StandardHours -- # of hours. All values are 80; so it is best to ignore this attribute.
* StockOptionLevel -- Stock grant issued during last review.
* TotalWorkingYears -- Total years of experience. Is maturity, mid-life crisis, old age a factor in attrition?
* TrainingTimesLastYear -- how much time did employee spend training on new skills last year. Is the academic pursuit and passion still existent in the employee?
* WorkLifeBalance -- is there a good work-life balance? Higher is better.
* YearsAtCompany -- how long in the current company have employees stayed?
* YearsInCurrentRole -- how long in current role; or other way of saying when did employee last change their role?
* YearsSinceLastPromotion -- how long since the employee been promoted?
* YearsWithCurrManager -- years under current manager. Indicates a level of comfort with management line.


In [ ]:
# Here is the data definitions from the Excel spreadsheet
pd.read_excel(uri, sheetname=1, names=['Attribute', 'Legend']).fillna('').T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
Attribute,Education,,,,,,EnvironmentSatisfaction,,,,,JobInvolvement,,,,,JobSatisfaction,,,,,PerformanceRating,,,,,RelationshipSatisfaction,,,,,WorkLifeBalance,,,
Legend,1 'Below College',2 'College',3 'Bachelor',4 'Master',5 'Doctor',,1 'Low',2 'Medium',3 'High',4 'Very High',,1 'Low',2 'Medium',3 'High',4 'Very High',,1 'Low',2 'Medium',3 'High',4 'Very High',,1 'Low',2 'Good',3 'Excellent',4 'Outstanding',,1 'Low',2 'Medium',3 'High',4 'Very High',,1 'Bad',2 'Good',3 'Better',4 'Best'


### 2.1 Visualization

Visualize the features to see if they make sense. You want good arity, correlation of the data

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.style.use('ggplot')

import math
from ggplot import *
fig = plt.figure(figsize=(20, 80))
cols = 3
label_col = 'Attrition'

# Draw correlation plots per column
rows = math.ceil(float(attrition_pd.shape[1]) / cols)
for i, column in enumerate(attrition_pd.columns):
    if column.lower() == label_col.lower():
        continue
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(column)
    if attrition_pd.dtypes[column] == np.object:
        cts = attrition_pd[[label_col, column]]
        cts = cts.groupby([label_col, column]).size()
        cts.unstack().T.plot(kind='bar', ax=ax, stacked=True, alpha=0.5)
    else:
        cts = attrition_pd[[label_col, column]]
        (xmin, xmax) = (min(cts[column].tolist()), max(cts[column].tolist()))
        cts.groupby(label_col)[column].plot(
            bins=16,
            kind='hist',
            stacked=True,
            alpha=0.5,
            legend=True,
            ax=ax,
            range=[xmin, xmax])

# Display plots
plt.subplots_adjust(hspace=0.7, wspace=0.2)

## 2.1 Feature Selection
By looking at the plots, and tallying with the hypothesis, can we validate we are along the right direction?

Here is notable visual observations we find (we will confirm statistically later) --
1. Age -- Older individuals ~55 years old do not attrite much.
2. DistanceFromHome -- longer commutes ~25 miles seem to attrite.
3. Education -- Individuals with doctoral degree do not attrite.
4. EmployeeCount, Over18, StandardHours -- they all exhibit an arity of 1 with no variance. These attributes are rubbish since we do not have any negative/variant examples.
5. EmployeeNumber -- is an ID attribute and useless as hypothesized
6. All satisfaction scores, as rightly hypothesized, exhibit higher attrition when satisfaction scores are lower. We still need a way to consolidate the three scores into one if possible.
7. All rates (daily, hourly, monthly) do not seem to show substantive patterns.
8. Singles show higher attrition as they churn faster.
9. Lower involvement rates seem to attrite more.
10. Sales and technicians attrite relatively more than other roles.
11. Lower monthly incomes -- no surprises -- do attrite and move to other opportunities.
12. NumCompaniesWorked -- people that have only had one or two jobs want to be adventurous.
13. Overtime -- people that are eligible for overtime (non exempt (fulltime?)) certainly seem to exhibit higher attrition pattern.
14. PercentSalaryHike -- people that received lower raises of course tend to move on.
15. Peformance Rating -- people with outstanding performance are recognized and rewarded; therefore attrite less. But we only see employees receiving excellent and outstanding ratings despite a wider range. So we have to renormalize the scale for better representation. Since the rating is already coded an ordinal attribute, we may be ok leaving as is too.
16. Lower stock option levels lead to higher attrition.
17. TotalWorkingYears -- people with approximately &lt;10 years attrite more. This variable is indeed confounded by age, marital status, experience, and YearsAtCompany perhaps!
18. TrainingTimesLastYear -- employees that do not seek training (are too snobby or too unskilled); and ones with 2-3 trainings seem to exhibit churn as skills are either too stale or improved beyond being applicable to current role.
19. YearsAtCompany and YearsInCurrentRole -- shorter experience window, especially in the first two years, seems to suggest quick attrition. This is understandable as new roles/employees are disconnected from business and usually take time to familiarize with the organization, business dynamics.
20. YearsSinceLastPromotion -- people with relatively long time since recent promotions seem to attrite less; perhaps they are just coasting in the current job.
21. YearsWithCurrentManager -- employees develop a trust with management in ~5 years. Before that, the lack of trust perhaps leads to attrition because of a clear alignment with management. Some individuals perhaps seek a change after a long term trust relationship too; so a few attrites do occur at the sixth year.

In [ ]:
# Aligned with our few observations, let us take some actions and weed out useless attributes
df = attrition_pd.copy()
#Remove ID attributes and the attributes that do not have a variance
df.drop('EmployeeCount', axis=1, inplace=True)
df.drop('EmployeeNumber', axis=1, inplace=True)
df.drop('Over18', axis=1, inplace=True)
df.drop('StandardHours', axis=1, inplace=True)

## 3. Model Building
Our exploration has given us a chance to gauge a few attributes and their characteristics. We have imputed and filtered values as needed. In our case, we did not have to impute, filter, or transform values.

Of the many attributes that seem to impact the attrition outcome, we do not know which is most discerning predictor. To discern, let us build a model...

We will be using a simple decision tree to investigate the predictor strength. We choose decision tree because we want explainability and palatibility of our model. Since decision tree works perfectly fine with both categorical, ordinal, and continuous data, we are good so far with the data. We may still need to transform the data however -- like map jan to 01, Feb to 02 etc depending on which implementation we choose. 

Alas, sklearn does require us to map categorical variables into one-hot encoded floats aka trump -> [1, 0] and clinton -> [0, 1] etc. Numerics and ordinals are fine.

In [ ]:
# Let us one-hot encode the data
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import MinMaxScaler

#Split the testset into a 90-10 split for training and testing
# Also remove all binary attributes and scale ordinal variables into 0-1 range
one_hot_encoded = pd.get_dummies(df).drop(
    label_col + '_No',
    axis=1).rename(columns={label_col + '_Yes': label_col}).drop(
        'OverTime_No', axis=1).rename(columns={'OverTime_Yes': 'OverTime'})
train, test = train_test_split(one_hot_encoded.copy(), test_size=0.1)

# Display the training set
display(train)

In [ ]:
# Define shortcuts for separating X and Y from pandas dataframe
X_set = lambda df: df.drop([label_col], axis=1)
Y_set = lambda df: df[label_col]

#Let us separate the training X and test X
X_train, y_train, X_test, y_test = (X_set(train), Y_set(train), X_set(test), Y_set(test))

In [ ]:
#Let us use Gradient Boosted Tree Classifier (prediction of a yes/no attrition) model -- a decision tree implementation
from sklearn import ensemble
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import *

# Fit classifier
params = {'n_estimators': 400, 'max_depth': 5}

# Create classifier
clf = ensemble.GradientBoostingClassifier(learning_rate=0.1)

# Train
clf.fit(X_train, y_train)

# Predict on testset and verify accuracy
acc = clf.score(X_test, y_test)

# Print
display(HTML("<h3 align='center'>Accuracy is <u>{:.2f}%</u></h3>".format(acc*100)))

## Feature Importances

What features seem to be explaining the attrition most?

In [ ]:
%matplotlib inline

# Plot feature importance normalized to a 100% scale
importances = pd.DataFrame(
    zip(X_train.columns.values, 100 * clf.feature_importances_ /
        clf.feature_importances_.max()),
    columns=['Feature', 'Importance %']).sort_values(
        ['Importance %'], ascending=[False])

# Chart most important features predicting the attrition outcome
sns.set_palette("husl")
importances.head(20).plot(kind='barh', x='Feature', y='Importance %')

## 3.1 Is the model any good?

In [ ]:
#Let us see how we predicted. What false positives and true negatives did we yield...
from sklearn.metrics import confusion_matrix

# Predict the attrition outcomes on the test set
y_pred = clf.predict(X_test)

# Draw the confusion matrix
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=None):
    # Show the confusion matrix as an image
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    # Put labels on axis
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels, rotation=45)
    # Pack it together
    plt.tight_layout()
    
    # Render the DataFrame as a table for easy view
    cmpd = pd.DataFrame(cm, columns=labels)
    cmpd.index = labels
    display(HTML('<b align="center">Confusion matrix</b>'))
    display(cmpd)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')


# Labels
labels = [0, 1]
# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=labels)

# Show the confusion matrix
plt.figure()
plot_confusion_matrix(cm, labels=labels)
plt.show()

How do we interpret these results?
* If you penalize both false-positives (where the model predicted the employee will attrite when indeed they did not!) and true-negatives (where the model predicted the employee will not attrite when they indeed did!), then the model accuracy is operating with ~85% accuracy. Btw, in real world, it is more detrimental to safely assume that an employee will not attrite when they do -- and one would want to err towards caution -- therefore be more intolerant to true negatives.
* To read a confusion matrix heuristically, if you treat the visualization of a matrix as a matrix, the elements along the diagonal must be large numbers and there should not be any spillage outside the diagonal. Spillage will seep to non diagonal elements which are indication that model and the real-world do not align. So it should be "dark" along the diagonal and white along non-diagonals.
* In our case, we seem to be falling into the fallacy of predicting an employee will not attrite when they indeed do in 19 test cases, and that is not good. Predicting 3 employees as attriting when they do not is not so much a challenge if the company values employee retention.

## 4. Reading/explaining the model

In [ ]:
# What does our data decision tree look like...
# Since GBT is an ensemble of randomforests/decisiontrees, we will refit the data just to render on the screen
from IPython.display import Image
import pydot
from sklearn import tree
from sklearn.externals.six import StringIO
from sklearn import tree

dtree = tree.DecisionTreeClassifier(max_depth=3)
dtree = dtree.fit(X_train, y_train)

dot_data = StringIO()
with open("output.dot", "w") as output_file:
    tree.export_graphviz(
        dtree,
        out_file=output_file,
        filled=True,
        rounded=True,
        feature_names=X_train.columns.values,
        class_names=['Stays', 'Leaves'],
        special_characters=True)
from os import system
system('dot -Tpng -o dtree2.png output.dot')
Image("dtree2.png")
#Double click on the image if you care.

## 4.1 Interpreting the chart

What does the decision tree really tell us? Focus on leaf nodes that show class="Leaves"
* First, traversing the left-most branch, that full time employees with less than 2.5 years of experience and those who make <= 58.5 predominantly leave.
* Second, traversing the left most branch of the right side, non-exempt young (<=30 years) employees with low income (< $2475) leave as well.

## 5. Scoring

Now that we have a model, one that performs ~85% accuracy, can we now predict for new employees (or current employees as they age) if/when they attrite? YES!

In [ ]:
#For the purposes of this demo, let us assume test data is indeed our scoring data. You want to ideally score on an unseen data...

#Predict the attrition likelihood
y_pred = clf.predict(X_test)
emit = X_test.copy()
emit['Really Attrited'] = y_test
emit['Predicted Attrited'] = y_pred

#Show the likelihoods
display(emit)

## Conclusion

A simple attrition analysis model. This is a purely quantitative model that looks at attributes; prescient systems thay_predt predict employee churn or customer churn based on prior activities (telephone calls, HR escalations, paystub downloads etc) also play a critical role in churn analysis.
<hr />